# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
file_path_list = []
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    files = glob.glob(os.path.join(root,'*'))
    for f in files:
        file_path_list.append(os.path.abspath(f))

# Remove any checkpoint csv file. There should be 30 file paths in total
to_remove = []
for file in file_path_list:
    if "checkpoint" in file:
        to_remove.append(file)
[file_path_list.remove(file) for file in to_remove]
print()

/home/workspace



#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        for line in csvreader:
            full_data_rows_list.append(line) 
print("Total Number of Rows = {}".format(len(full_data_rows_list)))
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


Total Number of Rows = 8056


# Part II. Apache Cassandra modelling

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [4]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [5]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity 
        WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as e:
    print(e)


#### Set Keyspace

In [6]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Create a mapping between column name and column index for the ease of creating tables and queries

In [7]:
column_dict = {"artist": 0, "first_name": 1, "gender": 2, "item_in_session": 3, "last_name": 4, "length": 5, 
               "level": 6, "location": 7, "session_id": 8, "song_title": 9, "user_id": 10}
file = 'event_datafile_new.csv'

### Query 1: Select the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

<strong>Interpretation of query:</strong> Query 1 aims to select artist name, song title, and the length of the correspnoding song in the app histry based on a session id and the item number in the session.

<strong>Expected Output</strong>
<img src="images/query_1_expected_output.png">

<strong>Decision of keys:</strong> session_id and item_in_session together can uniquely identify a record in the table, hence the primary key for this table should be a composite key, i.e. <code>PRIMARY KEY((session_id, item_in_session))</code>

<strong>Column Names</strong>:
* session_id: the id of a session, data type = int
* item in session: a unique identifier that identifies a record in a certain session, data type = int
* artist: the name of the artist, data type = text
* song_title: the name of the song, data type = text
* song_length: the length of the song in the form of seconds, data type = float

Step 1 for Query 1: Create a table called <code>song_by_session</code>

In [8]:
CREATE_TABLE_QUERY_1 = """CREATE TABLE IF NOT EXISTS song_by_session (session_id int, item_in_session int, 
                    artist text, song_title text, length float, PRIMARY KEY((session_id, item_in_session)))"""
try:
    session.execute(CREATE_TABLE_QUERY_1)
except Exception as e:
    print(e)

Step 2 for Query 1: Insert records into <code>song_by_session</code>

In [9]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_by_session (session_id, item_in_session, artist, song_title, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[column_dict["session_id"]]), int(line[column_dict["item_in_session"]]), line[column_dict["artist"]], 
                                line[column_dict["song_title"]], float(line[column_dict["length"]])))

Step 3 for Query 1: Run query 1 to verify the results

In [10]:
QUERY_1 = """SELECT artist, song_title, length FROM song_by_session WHERE session_Id = 338 AND item_in_session = 4"""
try:
    rows = session.execute(QUERY_1)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query 2: Select only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

<strong>Interpretation of query:</strong> Query 2 aims to select the artist name, the song title, and the user's first and last name by a given user id and a session id, the result will need to be sorted by the item_in_session attribute

<strong>Expected Output</strong>
<img src="images/query_2_expected_output.png">

<strong>Decision of keys:</strong> user_id and session_id together can uniquely identify a record in the table, hence the primary key for this table should be a composite key. item_in_session should be a clustering key to allow sorting action. Therefore, <code>PRIMARY KEY((session_id, item_in_session), item_in_session)</code>


<strong>Column Names</strong>:
* user_id: the id of a user, data type = int
* session_id: the id of a session, data type = int
* item in session: a unique identifier that identifies a record in a certain session, data type = int
* artist: the name of the artist, data type = text
* song_title: the name of the song, data type = text
* first_name: the first name of a user, data type = text
* last_name: the last name of a user, data type = text

Step 1 for Query 2: create the second table (song_by_user_and_session)

In [11]:
CREATE_TABLE_QUERY_2 = """CREATE TABLE IF NOT EXISTS song_by_user_and_session (user_id int, session_id int, item_in_session int, artist text, 
                        song_title text, first_name text, last_name text, PRIMARY KEY((user_id, session_id), item_in_session))"""
try:
    session.execute(CREATE_TABLE_QUERY_2)
except Exception as e:
    print(e)            

Step 2 for Query 2: Insert records into song_by_user_and_session

In [12]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_by_user_and_session(user_id, session_id, item_in_session, artist, song_title, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[column_dict["user_id"]]), int(line[column_dict["session_id"]]), int(line[column_dict["item_in_session"]]), line[column_dict["artist"]], 
                                line[column_dict["song_title"]], line[column_dict["first_name"]], line[column_dict["last_name"]]))

Step 3 for Query 2: execute query 2 and verify results

In [13]:
QUERY_2 = """SELECT artist, song_title, first_name, last_name FROM song_by_user_and_session WHERE user_id = 10 AND session_id = 182;"""
try:
    rows = session.execute(QUERY_2)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song_title, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query 3: Select every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

<strong>Interpretation of query:</strong> query 3 aims to find out the full name of all users who listened to a certain song.

<strong>Expected Output</strong>
<img src="images/query_3_expected_output.png">

<strong>Decision of keys:</strong> Despite song title alone cannot uniquely identifies all records in the table, the query requires only the song title in the <code>WHERE</code>, making song title the PRIMARY Key. Since there could be multiple users listened to the same song, user_id needs to be added as a clustering key to help uniquely identify each record. Therefore, <code>PRIMARY KEY(song_title, user_id)</code>

<strong>Column Names</strong>:
* song_title: the name of the song, data type = text
* user_id: the id of a user, data type = int
* first_name: the first name of a user, data type = text
* last_name: the last name of a user, data type = text

Step 1 for Query 3: create the third table song_by_title

In [14]:
CREATE_TABLE_QUERY_3 = """CREATE TABLE IF NOT EXISTS song_by_title (song_title text, user_id int, first_name text, last_name text, PRIMARY KEY(song_title, user_id ))"""
try:
    session.execute(CREATE_TABLE_QUERY_3)
except Exception as e:
    print(e)    
                    

Step 2 for Query 3: Insert records into song_by_title

In [15]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_by_title(song_title, user_id, first_name,last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[column_dict["song_title"]], int(line[column_dict["user_id"]]), line[column_dict["first_name"]], line[column_dict["last_name"]]))

Step 3 for Query 3: execute query 3 on song_by_title to obtain every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [16]:
QUERY_3 = """SELECT first_name, last_name FROM song_by_title WHERE song_title = 'All Hands Against His Own';"""
try:
    rows = session.execute(QUERY_3)
except Exception as e:
    print(e)

for row in rows:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [ ]:
DROP_QUERY_1 = """DROP TABLE IF EXISTS song_by_session;"""
DROP_QUERY_2 = """DROP TABLE IF EXISTS song_by_user_and_session;"""
DROP_QUERY_3 = """DROP TABLE IF EXISTS song_by_title;"""

try:
    session.execute(DROP_QUERY_1)
except Exception as e:
    print(e)
try:
    session.execute(DROP_QUERY_2)
except Exception as e:
    print(e)
try:
    session.execute(DROP_QUERY_3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()